https://docs.python.org/zh-cn/3/library/collections.html

In [1]:
import collections

## 1. class collections.ChainMap(*maps)¶

一个 ChainMap 将多个字典或者其他映射组合在一起，创建一个单独的可更新的视图。 

注意，一个 ChainMap() 的迭代顺序是通过从后往前扫描所有映射来确定的:

In [13]:
x = {'a': 1, 'b': 2}
y = {'b': 10, 'c': 11}
z = {'d': 20, 'e': 21}

combined = collections.ChainMap(x, y, z)
for k, v in combined.items():
    print(k, v)

print(combined, f'b 的值为: {combined["b"]}')

d 20
e 21
b 2
c 11
a 1
ChainMap({'a': 1, 'b': 2}, {'b': 10, 'c': 11}, {'d': 20, 'e': 21}) b 的值为: 2


In [14]:
combined = collections.ChainMap(z, y, x)
for k, v in combined.items():
    print(k, v)

print(combined, f'b 的值为: {combined["b"]}')

a 1
b 10
c 11
d 20
e 21
ChainMap({'d': 20, 'e': 21}, {'b': 10, 'c': 11}, {'a': 1, 'b': 2}) b 的值为: 10


一个 ChainMap 通过引用合并底层映射。 所以，如果一个底层映射更新了，这些更改会反映到 ChainMap 。

In [23]:
z['f'] = 30
for k, v in combined.items():
    print(k, v)

print(combined)

d 20
e 21
f 66
b 10
c 11
a 1
ChainMap({'a': 1, 'f': 66}, {'b': 10, 'c': 11}, {'d': 20, 'e': 21, 'f': 30})


如果没有指定任何 maps，一个空字典会被作为 maps。这样，每个新链至少包含一个映射。

In [21]:
chain_map = collections.ChainMap()
chain_map

ChainMap({})

### (1). maps 

ChainMap 底层映射被存储在一个列表中。这个列表是公开的，可以通过 maps 属性存取和更新。

In [31]:
x = {'a': 1, 'b': 2}
y = {'b': 10, 'c': 11}

combined = collections.ChainMap(x, y)
print(combined.maps)
z = {'d': 20, 'e': 21}
combined.maps.append(z)
print(combined.maps)

[{'a': 1, 'b': 2}, {'b': 10, 'c': 11}]
[{'a': 1, 'b': 2}, {'b': 10, 'c': 11}, {'d': 20, 'e': 21}]


搜索查询底层映射，直到一个键被找到。不同的是，写，更新和删除只操作第一个映射。

In [33]:
print(combined)
combined.update({'b': 3, 'f': 66})
print(combined)
del combined['b']
print(combined)
print(combined['b'])
# 虽然 combined['b'] 有, 但是不在第一个 dict 中, 删除 combined['b'] 会报错
# del combined['b']

ChainMap({'a': 1, 'f': 66}, {'b': 10, 'c': 11}, {'d': 20, 'e': 21})
ChainMap({'a': 1, 'f': 66, 'b': 3}, {'b': 10, 'c': 11}, {'d': 20, 'e': 21})
ChainMap({'a': 1, 'f': 66}, {'b': 10, 'c': 11}, {'d': 20, 'e': 21})
10


ChainMap 类只更新链中的第一个映射，但lookup会搜索整个链。 然而，如果需要深度写和删除，也可以很容易的通过定义一个子类来实现它

In [37]:
from collections import ChainMap


class DeepChainMap(ChainMap):
    'Variant of ChainMap that allows direct updates to inner scopes'

    def __setitem__(self, key, value):
        for mapping in self.maps:
            if key in mapping:
                mapping[key] = value
                return
        self.maps[0][key] = value

    def __delitem__(self, key):
        for mapping in self.maps:
            if key in mapping:
                del mapping[key]
                return
        raise KeyError(key)

In [38]:
d = DeepChainMap({'zebra': 'black'}, {'elephant': 'blue'}, {'lion': 'yellow'})
d['lion'] = 'orange'         # update an existing key two levels down
d['snake'] = 'red'           # new keys get added to the topmost dict
del d['elephant']            # remove an existing key one level down
d                            # display result

DeepChainMap({'zebra': 'black', 'snake': 'red'}, {}, {'lion': 'orange'})

### (2). new_child(m=None, **kwargs)

返回一个新的 ChainMap，其中包含一个新的映射，后面跟随当前实例中的所有映射。 如果指定了 m，它会成为新的映射加在映射列表的前面；如果未指定，则会使用一个空字典，因此调用 d.new_child() 就等价于 ChainMap({}, *d.maps)。 如果指定了任何关键字参数，它们会更新所传入的映射或新的空字典。 此方法被用于创建子上下文，它可在不改变任何上级映射的情况下被更新。

In [35]:
chain_map = collections.ChainMap()
nested_chain_map = chain_map.new_child()
nested_chain_map

ChainMap({}, {})

In [34]:
x = {'a': 1, 'b': 2}
y = {'b': 10, 'c': 11}
z = collections.ChainMap(y, x)

print(z.maps)

xx = {'d': 55, 'e': 88}
z = z.new_child(xx)
print(z.maps)

[{'b': 10, 'c': 11}, {'a': 1, 'b': 2}]
[{'d': 55, 'e': 88}, {'b': 10, 'c': 11}, {'a': 1, 'b': 2}]


### (3). parents

属性返回一个新的 ChainMap 包含所有的当前实例的映射，除了第一个。这样可以在搜索的时候跳过第一个映射。

使用的场景类似在 nested scopes 嵌套作用域中使用 nonlocal 关键词。用例也可以类比内建函数 super() 。

一个 d.parents 的引用等价于 ChainMap(*d.maps[1:]) 。

In [44]:
x = {'a': 1, 'b': 2}
y = {'b': 10, 'c': 11}
z = collections.ChainMap(x, y)

print(z.parents)
print(z.maps)

ChainMap({'b': 10, 'c': 11})
[{'a': 1, 'b': 2}, {'b': 10, 'c': 11}]


## 2. class collections.Counter([iterable-or-mapping])

Counter 是 dict 的子类，用于计数 hashable 对象。它是一个多项集，元素存储为字典的键而它们的计数存储为字典的值。

计数可以是任何整数，包括零或负的计数值。它可以通过计数一个 iterable 中的元素来初始化

In [1]:
from collections import Counter

In [7]:
cnt = Counter()
for word in ['red', 'blue', 'red', 'green', 'blue', 'blue']:
    cnt[word] += 1

print(cnt)
print(Counter(['red', 'blue', 'red', 'green', 'blue', 'blue']))

Counter({'blue': 3, 'red': 2, 'green': 1})
Counter({'blue': 3, 'red': 2, 'green': 1})


In [8]:
c = Counter('gallahad')                 # a new counter from an iterable
print(c)
c = Counter({'red': 4, 'blue': 2})      # a new counter from a mapping
print(c)
c = Counter(cats=4, dogs=8)             # a new counter from keyword args
print(c)

Counter({'a': 3, 'l': 2, 'g': 1, 'h': 1, 'd': 1})
Counter({'red': 4, 'blue': 2})
Counter({'dogs': 8, 'cats': 4})


Counter 对象的接口类似于字典，不同的是，如果查询的键不在 Counter 中，它会返回一个 0 而不是引发一个 KeyError：

In [9]:
c['bacon']

0

计数对象支持相等性、子集和超集关系等富比较运算符: ==, !=, <, <=, >, >=。 

所有这些检测会将不存在的元素当作计数值为零，因此 Counter(a=1) == Counter(a=1, b=0) 将返回真值。

In [17]:
c = Counter(a=3, b=1)
d = Counter(a=1, b=2)
print(c + d)                       # add two counters together:  c[x] + d[x]
print(c - d)                       # subtract (keeping only positive counts)
print(c & d)                       # intersection:  min(c[x], d[x])
print(c | d)                       # union:  max(c[x], d[x])
print(c == d)                      # equality:  c[x] == d[x]
print(c <= d)                      # inclusion:  c[x] <= d[x]

Counter({'a': 4, 'b': 3})
Counter({'a': 2})
Counter({'a': 1, 'b': 1})
Counter({'a': 3, 'b': 2})
False
False


单目加和减（一元操作符）意思是从空计数器加或者减去。

In [18]:
c = Counter(a=2, b=-4)
print(+c)
print(-c)

Counter({'a': 2})
Counter({'b': 4})


### (1). elements()

返回一个迭代器，其中每个元素将重复出现计数值所指定次。 元素会按首次出现的顺序返回。 

如果一个元素的计数值小于一，elements() 将会忽略它。

In [11]:
c = Counter(a=4, b=2, c=0, d=-2)
list(c.elements())

['a', 'a', 'a', 'a', 'b', 'b']

### (2). most_common([n])

返回一个列表，其中包含 n 个最常见的元素及出现次数，按常见程度由高到低排序。

如果 n 被省略或为 None，most_common() 将返回计数器中的 所有 元素。 

计数值相等的元素按首次出现的顺序排序：

In [12]:
Counter('abracadabra').most_common(3)

[('a', 5), ('b', 2), ('r', 2)]

### (3). subtract([iterable-or-mapping])

减去一个 可迭代对象 或 映射对象 (或 counter) 中的元素。

类似于 dict.update() 但是是减去而非替换。输入和输出都可以是 0 或负数。

In [13]:
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(a=1, b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 3, 'b': 0, 'c': -3, 'd': -6})

### (4). total()

计算总计数值。

In [15]:
c = Counter(a=10, b=5, c=0, d=-2)
c.total()

13

In [16]:
Counter(a=3, b=0) == Counter(a=3)

True